In [57]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import skimage.io
import os
from IPython.display import Image

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [58]:
#Load in image data
arrays = []
directory_ls = []

ugly_count = 0
pretty_count = 0

directory = r'photos'
for root, subdirectories, files in os.walk(directory):
    for file in files:
        if file.endswith('.txt') or file.endswith('json') or file.endswith('.DS_Store'):
          continue
        arrays.append(skimage.io.imread(os.path.join(root, file)))
        directory_ls.append(os.path.join(root, file))
        if 'pretty' in os.path.join(root, file):
          pretty_count+=1
        elif 'ugly' in os.path.join(root, file):
          ugly_count+=1

master_data = np.stack(arrays)
master_data = master_data / 255.0

#Generate labels
labels = []
for i in range(len(master_data)):
  if i >= ugly_count: 
    labels.append([1])
  else:
    labels.append([0])

labels = np.array(labels)
#Cross validation sets
#train_images, test_images, train_labels, test_labels = train_test_split(master_data, labels, test_size=0.33, random_state=42)

In [59]:
#Shuffles between generating a feature map and then selecting the most prominent features
#This is a pretty standard CNN setup
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3))) #Change input shape when working with Adam's data; (n,n,3)
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [60]:
#Add dense layers
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2)) #Current dataset has 10 output classes. Make sure to change this when loading in Adam's data.

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 21, 21, 64)        36928     
_________________________________________________________________
flatten_3 (Flatten)          (None, 28224)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)               

In [61]:
#Compile and train
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(master_data, labels, epochs=10)

Epoch 1/10
228/228 [==============================] - 113s 491ms/step - loss: 0.5471 - accuracy: 0.7277
Epoch 2/10
228/228 [==============================] - 112s 491ms/step - loss: 0.4596 - accuracy: 0.7914
Epoch 3/10
228/228 [==============================] - 111s 486ms/step - loss: 0.4166 - accuracy: 0.8100
Epoch 4/10
228/228 [==============================] - 112s 492ms/step - loss: 0.3838 - accuracy: 0.8267
Epoch 5/10
228/228 [==============================] - 111s 486ms/step - loss: 0.3295 - accuracy: 0.8567
Epoch 6/10
228/228 [==============================] - 113s 495ms/step - loss: 0.2779 - accuracy: 0.8806
Epoch 7/10
228/228 [==============================] - 111s 485ms/step - loss: 0.2089 - accuracy: 0.9133
Epoch 8/10
228/228 [==============================] - 111s 486ms/step - loss: 0.1421 - accuracy: 0.9467
Epoch 9/10
228/228 [==============================] - 111s 485ms/step - loss: 0.1040 - accuracy: 0.9624
Epoch 10/10
228/228 [==============================] - 111s 487m

In [1]:
#Turn logits into interpretable probabilities 
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
#Save model and convert to .json
probability_model.save('saved_correct/final_model.h5')

!tensorflowjs_converter --input_format keras \
                       saved_correct/final_model.h5 \
                       saved_correct_js

NameError: ignored

In [65]:
#Predict and display results

arrays = []
directory_ls = []

directory = r'' #Add path to directory with images you want to predict on
for root, subdirectories, files in os.walk(directory):
    for file in files:
        if file.endswith('.txt') or file.endswith('json') or file.endswith('.DS_Store'):
          continue
        arrays.append(skimage.io.imread(os.path.join(root, file)))
        directory_ls.append(os.path.join(root, file))

test_data = np.stack(arrays)
test_data = test_data / 255.0

predictions = probability_model.predict(test_data)

#Print nth prediction probabilities and display respective image
for i in range(len(test_data)):
  display(Image(filename=directory_ls[i]))
  print("Prediction: " + str(np.argmax(predictions[i])))



Output hidden; open in https://colab.research.google.com to view.